In [1]:
import pandas as pd

cwd = '/Users/asifrezachowdhury/Library/CloudStorage/OneDrive-Personal/Academics/QUT Academics/Semester 2-2/IFN711 Project/Project'
data_folder = cwd + '/data/'

leaveChart = pd.read_csv(data_folder+'EMP_CALDAY_REPTT.csv' ,on_bad_lines='skip',dtype='unicode')
actionTimes = pd.read_csv(data_folder+'EMP_ACTIONS.CSV',on_bad_lines='skip',dtype='unicode')



In [15]:
leaveChart['CALDAY'] = pd.to_datetime(leaveChart['CALDAY'], format=' %d/%m/%Y')
actionTimes['CALDAY'] = pd.to_datetime(actionTimes['CALDAY'], format='%Y%m%d')
endPoint = leaveChart.groupby('EMPLOYEE_GID')['CALDAY'].max().reset_index()
endPoint = pd.merge(endPoint, endPoint, on=['EMPLOYEE_GID', 'CALDAY'], how='left').rename(columns={"CALDAY": "LIMIT"})
endPoint #people who left

,EMPLOYEE_GID,LIMIT
0,000A9D12DA81C5EA71665DA92C52BC9B8F3882A05A1AF3...,2023-02-14
1,0025A43B35C495E75592FF837E0C0B9C5DD7BD296F4D9E...,2018-11-04
2,0029744DD95F1C9E9F55905F41678BDE48BED959F6BF1F...,2023-11-03
3,002B4157AD81B9AABC42F03187DBA067C565E57ED72381...,2022-06-24
4,002DD4A2DFC5A165491EA5E1E4EC4E588323AC5A3FB95B...,2020-12-08
...,...,...
3597,FF563C818CDD4A1F90FB083DFB219C63C551DF040A7F32...,2023-01-26
3598,FFB9B0F2E30D87709D293B1374BD3B3756F00F6EA46D10...,2023-08-01
3599,FFC7B7BAE7E8D89DCF143E63C83392ADA48264BF55612B...,2022-12-23
3600,FFDDB36550677B1858D36736D07BAF4C3104DF7D1F8DB7...,2020-07-06


In [14]:
employeeMasked = pd.read_csv(data_folder+'EMP_ATTR_FULL.CSV',on_bad_lines='skip',dtype='unicode')

isIn9_1and10_1 = actionTimes[(actionTimes["ACTIONTYPE"].isin((9,10))) & (actionTimes["ACTIONREAS"] == 1)]["EMPLOYEE_GID"].drop_duplicates() #9,1 & 10,1
employeeMasked = employeeMasked.sort_values('_BIC_TERMDATE', ascending=True).drop_duplicates('EMPLOYEE_GID').sort_index()
employeeMasked = employeeMasked[(employeeMasked["_BIC_VOLUNTRY"].isnull()) & ~employeeMasked["_BIC_VOLUNTRY"].isin(isIn9_1and10_1)]
employeeMasked = employeeMasked[["EMPLOYEE_GID"]]
# employeeMasked

employeeMasked3602=employeeMasked.sample(n=3602)
employeeMasked3602.to_csv(data_folder+"stillWithCompany3602.csv")
employeeMasked3602
# employeeMasked.to_csv("stillWithCompany.csv")
# employeeMasked

,EMPLOYEE_GID
219665,9058646DF201F73CA39E9E339618BC6056A5DAF9A19B51...
133090,98D55B965A3037DCD069758ACBCAC2BE7CA3E2625E3DAB...
135892,673F42E45A249AF36004A89EDE682ABEE1A4D5B7DDA950...
144662,7565D81FA474B0A7DED8FECAEC68CDD0AED09FD1C482A8...
137009,F68DDD0BBCE3C079C81255F87C193DF4E9AAD54FA9EA8F...
...,...
166791,C023D600F9B356CDF347FD3FDF7A1821D429E80A02F9F9...
57838,42BEF68BBE44358E3183A2A8673B54C5DB1B8E62EEC295...
214202,819DA7E271F8BFAF89AC95F45B09E5BE72F9DE2A0513E6...
84232,0A54A91AB14DAB99BEAFC2C82632876DDE6120FF7038C1...


In [7]:
mergedEndpoints = pd.merge(leaveChart, endPoint, on='EMPLOYEE_GID', how='left')
mergedEndpoints['SUBTRACTED'] = (mergedEndpoints['CALDAY'] - mergedEndpoints['LIMIT'])
months6 = mergedEndpoints[mergedEndpoints['SUBTRACTED'] > pd.Timedelta("-182 days")] # 6 months
months12 = mergedEndpoints[((mergedEndpoints['SUBTRACTED'] > pd.Timedelta("-365 days")) & (mergedEndpoints['SUBTRACTED'] < pd.Timedelta("-182 days")))] # 12 months

In [8]:
filter6 = months6["EMPLOYEE_GID"].isin(employeeMasked3602["EMPLOYEE_GID"])
filter12 = months12["EMPLOYEE_GID"].isin(employeeMasked3602["EMPLOYEE_GID"])

In [10]:
inCompany6 = months6[filter6]
inCompany12 = months12[filter12]
inCompany6.to_csv(data_folder+"stillWithCompany_6m.csv")
inCompany12.to_csv(data_folder+"stillWithCompany_12m.csv")

In [11]:
inCompany6.dtypes

EMPLOYEE_GID             object
CALDAY           datetime64[ns]
REPTT                    object
LIMIT            datetime64[ns]
SUBTRACTED      timedelta64[ns]
dtype: object

### Plotting

In [22]:
!pip3 install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 7.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 3.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [23]:
!pip3 install plotly==5.14.1


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
